In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import yfinance as yf

In [36]:
stocks_list = ["ITSA4", "ALUP11","SUZB3"]#, "CCRO3", "BBSE3", "JBSS3", "ENBR3", "MRVE3", "ASAI3", "CRFB3", "TOTS3", "GGBR4", "TIMS3", "IRBR3", "USIM5", "BRFS3", "BBDC4", "BRAP4", "ABEV3", "UGPA3", "PETR3", "VBBR3", "BPAC11", "ITSA4", "BPAN4", "CSAN3", "DXCO3", "TAEE11", "PCAR3", "CIEL3", "RENT3", "PRIO3", "EGIE3", "HAPV3", "LWSA3", "LREN3", "FLRY3", "CPLE6", "RRRP3", "ALPA4", "CASH3", "COGN3", "ENGI11", "VALE3", "PETZ3", "SUZB3", "EMBR3", "MRFG3", "KLBN11", "AZUL4", "CMIN3", "AMER3", "ELET6", "BRKM5", "BEEF3", "CSNA3", "MGLU3", "SLCE3", "PETR4", "POSI3", "GOLL4", "SBSP3", "CYRE3", "EQTL3", "RDOR3", "GOAU4", "WEGE3", "ECOR3", "ARZZ3", "NTCO3", "CPFE3", "IGTI11", "EZTC3", "B3SA3", "CMIG4", "RAIL3", "RAIZ4", "HYPE3", "ITUB4", "ENEV3", "BBAS3", "VIIA3", "CVCB3", "SOMA3", "VIVT3", "MULT3", "SMTO3", "YDUQ3", "SANB11"]
stocks_list = [i + '.SA' for i in stocks_list]

In [37]:
# Validar tickers descontinuados
ticker_valid = yf.download(stocks_list, start = '2024-01-01', end = '2024-01-31', period = "1d")
ticker_valid = ticker_valid['Close']
ticker_valid.dropna(axis = 1, inplace=True)
stocks_list = ticker_valid.columns

[*********************100%***********************]  3 of 3 completed


# Funções Auxiliares

### Definindo o algoritmo para simular as operações

In [38]:
import math

# Create a function to round any number to the smalles multiple of 100
def round_down(x):
    return int(math.floor(x / 100.0)) * 100

def backtest_algorithm(
    df,
    capital_exposure,
    initial_capital):

    # List with the total capital after every operation
    total_capital = [initial_capital]

    # List with profits for every operation
    all_profits = [] 

    ongoing = False

    for i in range(0,len(df)):

        if ongoing == True:

            if (df["Open"][i] >= target) | (df["Open"][i] <= stop): 
                exit = df["Open"][i]

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["Low"][i] <= stop: 
                exit = stop

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["High"][i] >= target: 
                exit = target

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

        else:
            if ~(np.isnan(df["buy_price"][i])):
                entry = df["buy_price"][i]
                stop = df["stop"][i]
                
                if df["Low"][i] > stop: 
                    ongoing = True
                    risk = entry - stop
                    target = df["target"][i]
                    shares = round_down(capital_exposure / risk)

    return all_profits, total_capital

### Calculando a estatística e a curva de capital

In [39]:
def get_drawdown(data, column = "Close"):
    data["Max"] = data[column].cummax()
    data["Delta"] = data['Max'] - data[column]
    data["Drawdown"] = 100 * (data["Delta"] / data["Max"])
    max_drawdown = data["Drawdown"].max()
    return max_drawdown

def strategy_test(all_profits, total_capital):
    if len(all_profits) > 0:
        gains = sum(x >= 0 for x in all_profits)
        losses = sum(x < 0 for x in all_profits)
        num_operations = gains + losses
        pct_gains = 100 * (gains / num_operations)
        pct_losses = 100 - pct_gains
        total_profit = sum(all_profits)
        pct_profit = (total_profit / total_capital[0]) * 100
        
        # Compute drawdown
        total_capital = pd.DataFrame(data=total_capital, columns=["total_capital"])
        drawdown = get_drawdown(data=total_capital, column="total_capital")

        # Compute profit per operation
        profit_per_operation = pct_profit / num_operations

        # Expeceted Value
        all_positives = [x for x in all_profits if x >= 0]
        if len(all_positives) > 0:
            average_gain = sum(all_positives) / len(all_positives)
        else:
            average_gain = 0

        all_negatives = [x for x in all_profits if x < 0]
        if len(all_negatives) > 0:
            average_loss = sum(all_negatives) / len(all_negatives)
        else:
            average_loss = 0
            
        num_operations = len(all_profits)
        pct_gains = (len(all_positives) / num_operations)
        pct_losses = 1 - pct_gains
        
        expected_value = (average_gain * pct_gains) + (average_loss * pct_losses)
    else:
        total_profit = 0
        pct_profit = 0
        drawdown = 0
        num_operations = 0
        profit_per_operation = 0
        gains = 0
        pct_gains = 0
        losses = 0
        pct_losses = 0
        expected_value = 0

    return {
        "total_profit": total_profit,
        "pct_profit": pct_profit,
        "pct_drawdown": drawdown,
        "num_operations": num_operations,
        "pct_profit_per_operation": profit_per_operation,
        "num_gains": gains ,
        "pct_gains": pct_gains,
        "num_losses": losses,
        "pct_losses": pct_losses, 
        "expected_value": expected_value
    }
def capital_plot(total_capital, all_profits):
  all_profits = [0] + all_profits # make sure both lists are the same size
  cap_evolution = pd.DataFrame({'Capital': total_capital, 'Profit': all_profits})
  plt.title("Curva de Capital")
  plt.xlabel("Total Operações")
  cap_evolution['Capital'].plot()

# Ajuste do DF para chamada das Funções de modo Escalável

In [40]:
df_stats = pd.DataFrame()
for stock in stocks_list:
    df = yf.download(stock, start = '2010-01-01', interval = '1d')
    df.drop(['Adj Close','Volume'], axis=1, inplace=True)

### Definindo o candle sinal
    condition_1 = df["Low"] > df["Low"].shift(1)
    condition_2 = df["Low"].shift(1) < df["Low"].shift(2)
    df["signal"] = condition_1 & condition_2

### Calculando o Éden dos Traders
    df["mme8"] = df["Close"].ewm(span=8, min_periods=8).mean()
    df["mme80"] = df["Close"].ewm(span=80, min_periods=80).mean()
    df["eden"] = (df["mme8"] > df["mme8"].shift(1)) & (df["mme80"] > df["mme80"].shift(1)) #& (df["mme8"] > df["mme80"])
    df.dropna(inplace=True)

### Definindo o preço de compra
    condition_1 = df["signal"].shift(1) == True
    condition_2 = df["eden"].shift(1) == True 
    condition_3 = df["High"] > df["High"].shift(1)
    tick = 0.01

    df["buy_price"] = np.where(
        condition_1 & condition_2 & condition_3, 
        np.where(df["Open"] > df["High"].shift(1), df["Open"], df["High"].shift(1) + tick),
        np.nan)
    
### Definindo o alvo
    max_high = df["High"].rolling(3).max()
    min_low = df["Low"].rolling(3).min()
    amplitude = (max_high.shift(1) - min_low.shift(1))
    entry = df["High"].shift(1)
    df["target"] =  amplitude + entry

### Definindo o stop
    df["stop"] = df["Low"].shift(2) - tick

### Realizando o Backtest
    all_profits, total_capital = backtest_algorithm(
    df=df,
    capital_exposure=1000,
    initial_capital=100000)
    statistics = strategy_test(all_profits, total_capital)
    statistics = pd.DataFrame.from_dict(statistics, orient='index').round(2)
    df_stats[stock] = statistics
df_stats = df_stats.T

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [41]:
max_high

Date
2010-04-30          NaN
2010-05-03          NaN
2010-05-04    16.007999
2010-05-05    16.007999
2010-05-06    16.007999
                ...    
2024-03-05    58.980000
2024-03-06    58.980000
2024-03-07    58.980000
2024-03-08    59.270000
2024-03-11    59.540001
Name: High, Length: 3442, dtype: float64

In [42]:
df_stats.sort_values('total_profit').tail(20)

,total_profit,pct_profit,pct_drawdown,num_operations,pct_profit_per_operation,num_gains,pct_gains,num_losses,pct_losses,expected_value
ITSA4.SA,-12446.87,-12.45,17.74,103.0,-0.12,48.0,0.47,55.0,0.53,-120.84
ALUP11.SA,7586.13,7.59,5.46,82.0,0.09,46.0,0.56,36.0,0.44,92.51
SUZB3.SA,18731.00,18.73,3.02,50.0,0.37,34.0,0.68,16.0,0.32,374.62


In [43]:
df[df['buy_price'] > 0].tail().round(2)

,Open,High,Low,Close,signal,mme8,mme80,eden,buy_price,target,stop
Date,,,,,,,,,,,
2023-11-17,53.48,54.22,52.94,52.94,False,53.14,51.85,False,53.58,56.19,52.49
2023-12-20,52.70,54.59,52.70,53.90,False,52.64,52.12,True,53.44,54.83,52.05
2024-02-27,56.40,57.66,55.86,56.82,False,55.16,53.05,True,56.40,57.79,54.83
2024-03-04,57.39,58.55,57.39,58.45,False,56.69,53.46,True,57.64,59.43,56.23
2024-03-11,58.72,59.54,58.30,58.48,False,58.12,54.07,True,59.28,60.95,57.58
